In [42]:
%run model_sucrose.ipynb

Computing gene knockout strategies using the OptGene algorithm

### Preparations

Basing algorithms upon the surfactin reaction which is based in the fatty acid that is most utilized. Looking at the reaction stoichiometry again:

In [43]:
print(model.reactions.SP_1)

SP_1: 3hddcoa_c + asp__L_c + atp_c + glu__L_c + 4 leu__L_c + nadph_c + val__L_c --> adp_c + nadp_c + surfactin_c


Finding the other reaction metabolite names that are needed:

In [44]:
model.reactions.query("biomass","name") #Biomass reaction name

[<Reaction BIOMASS_BS_10 at 0x1cf9b238f10>]

In [45]:
model.metabolites.query("sucr") #Glucose metabolite name

[<Metabolite sucr_c at 0x1cf9142d430>, <Metabolite sucr_e at 0x1cf9142d580>]

Changing objective to surfactin

In [46]:
model.objective = model.reactions.SP_1

Import the packages which are necessary for OptGene:

In [47]:
import cameo #after pip install cameo in terminal

In [48]:
from cameo.strain_design.heuristic.evolutionary_based import OptGene

In [49]:
from cameo import models

### OptGene Algorithm

Running the OptGene algorithm to search for gene or reaction knockouts:

In [50]:
optgene = OptGene(model)

In [51]:
result_optgene = optgene.run(target="SP_1", #Reaction
                     biomass="BIOMASS_BS_10", #Reaction
                     substrate="sucr_e", #Metabolite
                     max_evaluations=20000, max_knockouts=3, plot=False)

Starting optimization at Wed, 24 Nov 2021 23:07:41


HBox()

Finished after 00:12:59


In [52]:
result_optgene

HBox()

,reactions,genes,size,fva_min,fva_max,target_flux,biomass_flux,yield,fitness
0,"(CYTB_B3,)","((BSU38140,), (BSU38160,), (BSU38150,), (BSU38...",1,0.0,0.003571,0.0,0.624185,0.0,0.0
1,"(GART,)","((BSU02230,),)",1,0.0,0.003571,0.0,0.624185,0.0,0.0
2,"(CYTB_B3, CDGPT_BS)","((BSU38160, BSU16920),)",2,0.0,0.343651,0.0,0.023593,0.0,0.0
3,"(CYTB_B3, UGT2_BS, UGT_BS, UGT1_BS)","((BSU38160, BSU21920),)",2,0.0,0.343740,0.0,0.023593,0.0,0.0
4,"(UGT1_BS, UGT2_BS, UGT_BS, CYTB_B3)","((BSU21920, BSU38170),)",2,0.0,0.343740,0.0,0.023593,0.0,0.0
5,"(UGT1_BS, UGT_BS, CYTB_B3, CDGPT_BS, UGT2_BS)","((BSU38160, BSU16920, BSU21920),)",3,0.0,0.343740,0.0,0.023593,0.0,0.0


No optimization suggestions were obtained from Optgene. 

Double checking this result by manually knocking out this reaction and finding surfactin yield:

In [63]:
with model as mutant0:
    mutant0.reactions.CYTB_B3.bounds = 0, 0
    print("LDH_L's bounds inside the with statement:")
    print(mutant0.reactions.CYTB_B3.lower_bound, mutant0.reactions.CYTB_B3.bounds)
    print('Mutant growth rate [mmol gDW^-1 h^-1]: ', mutant0.optimize().objective_value)
    mutant0.objective = mutant0.reactions.SP_1
    surfactin_production = mutant0.optimize().objective_value
    print("Theoretical max. yield [mmol-succ / mmol-ac]:", surfactin_production / (-1*mutant0.reactions.EX_sucr_e.flux))


LDH_L's bounds inside the with statement:
0 (0, 0)
Mutant growth rate [mmol gDW^-1 h^-1]:  0.3436433767512144
Theoretical max. yield [mmol-succ / mmol-ac]: 0.12073337894167445


In [66]:
with model as mutant1:
    mutant1.genes.BSU02230.knock_out()
    print("Flux:",mutant1.slim_optimize())
    mutant1.objective = mutant1.reactions.SP_1
    surfactin_production = mutant1.optimize().objective_value
    print("Max. surfactin production [mmol gDW^-1 h^-1]:", surfactin_production)
    print("Theoretical max. yield [mmol-surfactin / mmol-sucrose]:", surfactin_production / (-1*mutant1.reactions.EX_sucr_e.flux))

Flux: 0.3436433767512144
Max. surfactin production [mmol gDW^-1 h^-1]: 0.3436433767512144
Theoretical max. yield [mmol-surfactin / mmol-sucrose]: 0.12073337894167445


In [30]:
with model as mutant2:
    mutant2.genes.BSU21920.knock_out()
    mutant2.genes.BSU16920.knock_out()
    print("Flux:", mutant2.slim_optimize())
    surfactin_production = model.optimize().objective_value
    print("Max. surfactin production [mmol gDW^-1 h^-1]:", surfactin_production)
    print("Theoretical max. yield [mmol-surfactin / mmol-sucrose]:", surfactin_production / (-1*model.reactions.EX_glc__D_e.flux))

Flux: 0.07568534919274368
Max. surfactin production [mmol gDW^-1 h^-1]: 0.07568534919274368
Theoretical max. yield [mmol-surfactin / mmol-sucrose]: 0.0445207936427904


In [25]:
with model:
    model.reactions.FDH.bounds = 0, 0
    print("IPDDI's bounds inside the with statement")
    print(model.reactions.FDH.lower_bound, model.reactions.FDH.bounds)
    print('Mutant growth rate [mmol gDW^-1 h^-1]: ', model.optimize().objective_value)
    with model:
        model.objective = model.reactions.SP_1
        surfactin_production = model.optimize().objective_value
        print("Max. surfactin production [mmol gDW^-1 h^-1]:", surfactin_production)
        print("Theoretical max. yield [mmol-surfactin / mmol-sucrose]:", surfactin_production / (-1*model.reactions.EX_sucr_e.flux))


IPDDI's bounds inside the with statement
0 (0, 0)
Mutant growth rate [mmol gDW^-1 h^-1]:  0.07461282448997686
Max. surfactin production [mmol gDW^-1 h^-1]: 0.07461282448997686


ZeroDivisionError: float division by zero

It seems that the OptKnock algorithm has suggested a few knockoutS that leads to an increase in yield.